## Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install scikit-optimize

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 2.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/UTMW3.csv')

In [ ]:
df = df.drop('OwnerUserId', axis=1)
print(df.head())

   proof  probability  finite-automata  np-complete  \
0      0            0                0            0   
1      0            0                0            0   
2      0            0                0            0   
3      0            0                0            0   
4      0            0                0            0   

   mathematical-optimization  linear-programming  satisfiability  \
0                          0                   0               0   
1                          0                   0               0   
2                          0                   0               0   
3                          0                   0               0   
4                          0                   0               0   

   stable-marriage  np  complexity-theory  ...  list  singly-linked-list  \
0                0   0                  0  ...     0                   0   
1                0   0                  0  ...     0                   0   
2                0   0          

In [ ]:
import numpy as np
import tensorflow as tf
import logging
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import mean_squared_error

tf.compat.v1.disable_eager_execution()
log = logging.getLogger(__name__)

class RBM:
    """Restricted Boltzmann Machine"""

    def __init__(
        self,
        visible_units,
        hidden_units=500,
        keep_prob=0.7,
        init_stdv=0.1,
        learning_rate=0.004,
        minibatch_size=100,
        training_epoch=20,
        display_epoch=10,
        sampling_protocol=[50, 70, 80, 90, 100],
        debug=False,
        with_metrics=False,
        seed=42,
    ):
        self.n_hidden = hidden_units
        self.keep = keep_prob
        self.stdv = init_stdv
        self.learning_rate = learning_rate
        self.minibatch = minibatch_size
        self.epochs = training_epoch + 1
        self.display_epoch = display_epoch
        self.sampling_protocol = sampling_protocol
        self.debug = debug
        self.with_metrics = with_metrics
        self.seed = seed
        np.random.seed(self.seed)
        tf.compat.v1.set_random_seed(self.seed)
        self.n_visible = visible_units

        tf.compat.v1.reset_default_graph()

        self.generate_graph()
        self.init_metrics()
        self.init_gpu()
        init_graph = tf.compat.v1.global_variables_initializer()

        self.sess = tf.compat.v1.Session(config=self.config_gpu)
        self.sess.run(init_graph)

    def binomial_sampling(self, pr):
        g = tf.convert_to_tensor(value=np.random.uniform(size=pr.shape[1]), dtype=tf.float32)
        h_sampled = tf.nn.relu(tf.sign(pr - g))
        return h_sampled

    def free_energy(self, x):
        bias = -tf.reduce_sum(input_tensor=tf.matmul(x, tf.transpose(a=self.bv)))
        phi_x = tf.matmul(x, self.w) + self.bh
        f = -tf.reduce_sum(input_tensor=tf.nn.softplus(phi_x))
        F = bias + f
        return F

    def placeholder(self):
        self.vu = tf.compat.v1.placeholder(shape=[None, self.n_visible], dtype="float32")

    def init_parameters(self):
        with tf.compat.v1.variable_scope("Network_parameters"):
            self.w = tf.compat.v1.get_variable(
                "weight",
                [self.n_visible, self.n_hidden],
                initializer=tf.compat.v1.random_normal_initializer(stddev=self.stdv, seed=self.seed),
                dtype="float32",
            )
            self.bv = tf.compat.v1.get_variable(
                "v_bias",
                [1, self.n_visible],
                initializer=tf.compat.v1.zeros_initializer(),
                dtype="float32",
            )
            self.bh = tf.compat.v1.get_variable(
                "h_bias",
                [1, self.n_hidden],
                initializer=tf.compat.v1.zeros_initializer(),
                dtype="float32",
            )

    def sample_hidden_units(self, vv):
        phi_v = tf.matmul(vv, self.w) + self.bh
        phv = tf.nn.sigmoid(phi_v)
        phv_reg = tf.nn.dropout(phv, 1 - (self.keep))
        h_ = self.binomial_sampling(phv_reg)
        return phv, h_

    def sample_visible_units(self, h):
        phi_h = tf.matmul(h, tf.transpose(a=self.w)) + self.bv
        v_ = tf.nn.sigmoid(phi_h)
        return v_

    def gibbs_sampling(self):
        self.v_k = self.v
        for _ in range(self.k):
            _, h_k = self.sample_hidden_units(self.v_k)
            self.v_k = self.sample_visible_units(h_k)

    def losses(self, vv):
        with tf.compat.v1.variable_scope("losses"):
            obj = self.free_energy(vv) - self.free_energy(self.v_k)
        return obj

    def gibbs_protocol(self, i):
        with tf.compat.v1.name_scope("gibbs_protocol"):
            epoch_percentage = (i / self.epochs) * 100
            if epoch_percentage != 0:
                if epoch_percentage >= self.sampling_protocol[self.l] and epoch_percentage <= self.sampling_protocol[self.l + 1]:
                    self.k += 1
                    self.l += 1
                    self.gibbs_sampling()

    def data_pipeline(self):
        self.batch_size = tf.compat.v1.placeholder(tf.int64)
        self.dataset = tf.data.Dataset.from_tensor_slices(self.vu)
        self.dataset = self.dataset.shuffle(buffer_size=50, reshuffle_each_iteration=True, seed=self.seed)
        self.dataset = self.dataset.batch(batch_size=self.batch_size).repeat()
        self.iter = tf.compat.v1.data.make_initializable_iterator(self.dataset)
        self.v = self.iter.get_next()

    def init_metrics(self):
        if self.with_metrics:
            self.rmse = tf.sqrt(tf.compat.v1.losses.mean_squared_error(self.v, self.v_k, weights=tf.where(self.v > 0, 1, 0)))

    def generate_graph(self):
        log.info("Creating the computational graph")
        self.placeholder()
        self.data_pipeline()
        self.init_parameters()
        log.info("Initialize Gibbs protocol")
        self.k = 1
        self.l = 0
        self.gibbs_sampling()
        obj = self.losses(self.v)
        rate = self.learning_rate / self.minibatch
        self.opt = tf.compat.v1.train.AdamOptimizer(learning_rate=rate).minimize(loss=obj)

    def init_gpu(self):
        self.config_gpu = tf.compat.v1.ConfigProto(log_device_placement=False, allow_soft_placement=True)
        self.config_gpu.gpu_options.allow_growth = True

    def init_training_session(self, xtr):
        self.sess.run(self.iter.initializer, feed_dict={self.vu: xtr, self.batch_size: self.minibatch})
        self.sess.run(tf.compat.v1.tables_initializer())

    def batch_training(self, num_minibatches):
        epoch_tr_err = 0
        for _ in range(num_minibatches):
            if self.with_metrics:
                _, batch_err = self.sess.run([self.opt, self.rmse])
                epoch_tr_err += batch_err / num_minibatches
            else:
                _ = self.sess.run(self.opt)
        return epoch_tr_err

    def fit(self, xtr):
        self.init_training_session(xtr)
        for epoch in range(self.epochs):
            self.gibbs_protocol(epoch)
            tr_err = self.batch_training(num_minibatches=len(xtr) // self.minibatch)
            if epoch % self.display_epoch == 0 or epoch == self.epochs - 1:
                log.info(f"Epoch {epoch}: Training error: {tr_err}")

    def predict(self, xtst):
        self.sess.run(self.iter.initializer, feed_dict={self.vu: xtst, self.batch_size: xtst.shape[0]})
        preds = self.sess.run(self.v_k)
        return preds

def hyperparameter_tuning(data, param_grid, test_size=0.2, random_state=42):
    X_train, X_test = train_test_split(data, test_size=test_size, random_state=random_state)
    best_params = None
    best_rmse = float('inf')
    for params in ParameterGrid(param_grid):
        model = RBM(visible_units=data.shape[1], **params)
        model.fit(X_train)
        predictions = model.predict(X_test)
        log.info(f"Shape of X_test: {X_test.shape}, Shape of predictions: {predictions.shape}")
        rmse = np.sqrt(mean_squared_error(X_test[X_test > 0], predictions[X_test > 0]))
        log.info(f"Params: {params} - RMSE: {rmse}")
        if rmse < best_rmse:
            best_rmse = rmse
            best_params = params
    log.info(f"Best Params: {best_params} - Best RMSE: {best_rmse}")
    return best_params, best_rmse

# Example Usage
if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO)
    import pandas as pd

    # Example data, replace with actual dataset
    data = df

    param_grid = {
        'hidden_units': [50, 100],
        'keep_prob': [0.7, 0.9],
        'learning_rate': [0.01, 0.005],
        'minibatch_size': [50, 100],
        'training_epoch': [20, 50]
    }
    best_params, best_rmse = hyperparameter_tuning(data.values, param_grid)
    print(f"Best Params: {best_params} - Best RMSE: {best_rmse}")


Best Params: {'hidden_units': 100, 'keep_prob': 0.9, 'learning_rate': 0.005, 'minibatch_size': 50, 'training_epoch': 20} - Best RMSE: 274.20690044242235


In [ ]:
import numpy as np
import tensorflow as tf
import logging
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import ndcg_score
import pandas as pd
from tqdm import tqdm

tf.compat.v1.disable_eager_execution()
log = logging.getLogger(__name__)

class RBM:
    """Restricted Boltzmann Machine for Recommendation System"""

    def __init__(
        self,
        visible_units,
        hidden_units=500,
        keep_prob=0.7,
        init_stdv=0.1,
        learning_rate=0.004,
        minibatch_size=100,
        training_epoch=20,
        display_epoch=10,
        sampling_protocol=[50, 70, 80, 90, 100],
        debug=False,
        with_metrics=True,
        seed=42,
    ):
        self.n_hidden = hidden_units
        self.keep = keep_prob
        self.stdv = init_stdv
        self.learning_rate = learning_rate
        self.minibatch = minibatch_size
        self.epochs = training_epoch + 1
        self.display_epoch = display_epoch
        self.sampling_protocol = sampling_protocol
        self.debug = debug
        self.with_metrics = with_metrics
        self.seed = seed
        np.random.seed(self.seed)
        tf.compat.v1.set_random_seed(self.seed)
        self.n_visible = visible_units

        tf.compat.v1.reset_default_graph()

        self.generate_graph()
        self.init_metrics()
        self.init_gpu()
        init_graph = tf.compat.v1.global_variables_initializer()

        self.sess = tf.compat.v1.Session(config=self.config_gpu)
        self.sess.run(init_graph)

    def binomial_sampling(self, pr):
        g = tf.convert_to_tensor(value=np.random.uniform(size=pr.shape[1]), dtype=tf.float32)
        h_sampled = tf.nn.relu(tf.sign(pr - g))
        return h_sampled

    def free_energy(self, x):
        bias = -tf.reduce_sum(input_tensor=tf.matmul(x, tf.transpose(a=self.bv)))
        phi_x = tf.matmul(x, self.w) + self.bh
        f = -tf.reduce_sum(input_tensor=tf.nn.softplus(phi_x))
        F = bias + f
        return F

    def placeholder(self):
        self.vu = tf.compat.v1.placeholder(shape=[None, self.n_visible], dtype="float32")

    def init_parameters(self):
        with tf.compat.v1.variable_scope("Network_parameters"):
            self.w = tf.compat.v1.get_variable(
                "weight",
                [self.n_visible, self.n_hidden],
                initializer=tf.compat.v1.random_normal_initializer(stddev=self.stdv, seed=self.seed),
                dtype="float32",
            )
            self.bv = tf.compat.v1.get_variable(
                "v_bias",
                [1, self.n_visible],
                initializer=tf.compat.v1.zeros_initializer(),
                dtype="float32",
            )
            self.bh = tf.compat.v1.get_variable(
                "h_bias",
                [1, self.n_hidden],
                initializer=tf.compat.v1.zeros_initializer(),
                dtype="float32",
            )

    def sample_hidden_units(self, vv):
        phi_v = tf.matmul(vv, self.w) + self.bh
        phv = tf.nn.sigmoid(phi_v)
        phv_reg = tf.nn.dropout(phv, 1 - (self.keep))
        h_ = self.binomial_sampling(phv_reg)
        return phv, h_

    def sample_visible_units(self, h):
        phi_h = tf.matmul(h, tf.transpose(a=self.w)) + self.bv
        v_ = tf.nn.sigmoid(phi_h)
        return v_

    def gibbs_sampling(self):
        self.v_k = self.v
        for _ in range(self.k):
            _, h_k = self.sample_hidden_units(self.v_k)
            self.v_k = self.sample_visible_units(h_k)

    def losses(self, vv):
        with tf.compat.v1.variable_scope("losses"):
            obj = self.free_energy(vv) - self.free_energy(self.v_k)
        return obj

    def gibbs_protocol(self, i):
        with tf.compat.v1.name_scope("gibbs_protocol"):
            epoch_percentage = (i / self.epochs) * 100
            if epoch_percentage != 0:
                if epoch_percentage >= self.sampling_protocol[self.l] and epoch_percentage <= self.sampling_protocol[self.l + 1]:
                    self.k += 1
                    self.l += 1
                    self.gibbs_sampling()

    def data_pipeline(self):
        self.batch_size = tf.compat.v1.placeholder(tf.int64)
        self.dataset = tf.data.Dataset.from_tensor_slices(self.vu)
        self.dataset = self.dataset.shuffle(buffer_size=50, reshuffle_each_iteration=True, seed=self.seed)
        self.dataset = self.dataset.batch(batch_size=self.batch_size).repeat()
        self.iter = tf.compat.v1.data.make_initializable_iterator(self.dataset)
        self.v = self.iter.get_next()

    def init_metrics(self):
        if self.with_metrics:
            self.rmse = tf.sqrt(tf.compat.v1.losses.mean_squared_error(self.v, self.v_k, weights=tf.where(self.v > 0, 1, 0)))

    def generate_graph(self):
        log.info("Creating the computational graph")
        self.placeholder()
        self.data_pipeline()
        self.init_parameters()
        log.info("Initialize Gibbs protocol")
        self.k = 1
        self.l = 0
        self.gibbs_sampling()
        obj = self.losses(self.v)
        rate = self.learning_rate / self.minibatch
        self.opt = tf.compat.v1.train.AdamOptimizer(learning_rate=rate).minimize(loss=obj)

    def init_gpu(self):
        self.config_gpu = tf.compat.v1.ConfigProto(log_device_placement=False, allow_soft_placement=True)
        self.config_gpu.gpu_options.allow_growth = True

    def init_training_session(self, xtr):
        self.sess.run(self.iter.initializer, feed_dict={self.vu: xtr, self.batch_size: self.minibatch})
        self.sess.run(tf.compat.v1.tables_initializer())

    def batch_training(self, num_minibatches):
        epoch_tr_err = 0
        for _ in range(num_minibatches):
            if self.with_metrics:
                _, batch_err = self.sess.run([self.opt, self.rmse])
                epoch_tr_err += batch_err / num_minibatches
            else:
                _ = self.sess.run(self.opt)
        return epoch_tr_err

    def fit(self, xtr):
        self.init_training_session(xtr)
        for epoch in tqdm(range(self.epochs), desc="Training"):
            self.gibbs_protocol(epoch)
            tr_err = self.batch_training(num_minibatches=len(xtr) // self.minibatch)
            if epoch % self.display_epoch == 0 or epoch == self.epochs - 1:
                log.info(f"Epoch {epoch}: Training error: {tr_err}")

    def predict(self, xtst):
        self.sess.run(self.iter.initializer, feed_dict={self.vu: xtst, self.batch_size: xtst.shape[0]})
        preds = self.sess.run(self.v_k)
        return preds

def calculate_precision(y_true, y_pred, k=5):
    """
    Calculate Precision@k for each user.

    Args:
    y_true: True ratings (2D numpy array)
    y_pred: Predicted ratings (2D numpy array)
    k: Number of top items to consider

    Returns:
    Mean Precision@k across all users
    """
    precisions = []
    for user_true, user_pred in zip(y_true, y_pred):
        # Get indices of rated items in true ratings
        relevant_items = np.where(user_true > 0)[0]

        # If user has no relevant items, skip
        if len(relevant_items) == 0:
            continue

        # Get top k predicted items
        recommended_items = np.argsort(user_pred)[::-1][:k]

        # Calculate precision for this user
        hits = np.isin(recommended_items, relevant_items)
        precision = np.sum(hits) / k
        precisions.append(precision)

    return np.mean(precisions) if precisions else 0

def calculate_map(y_true, y_pred, k=5):
    """
    Calculate Mean Average Precision@k.

    Args:
    y_true: True ratings (2D numpy array)
    y_pred: Predicted ratings (2D numpy array)
    k: Number of top items to consider

    Returns:
    MAP@k
    """
    aps = []
    for user_true, user_pred in zip(y_true, y_pred):
        relevant_items = np.where(user_true > 0)[0]

        if len(relevant_items) == 0:
            continue

        recommended_items = np.argsort(user_pred)[::-1][:k]

        hits = np.isin(recommended_items, relevant_items)
        precisions = np.cumsum(hits) / (np.arange(len(hits)) + 1)
        ap = np.sum(precisions * hits) / min(k, len(relevant_items))
        aps.append(ap)

    return np.mean(aps) if aps else 0

def calculate_recall(y_true, y_pred, k=5):
    """
    Calculate Recall@k for each user.

    Args:
    y_true: True ratings (2D numpy array)
    y_pred: Predicted ratings (2D numpy array)
    k: Number of top items to consider

    Returns:
    Mean Recall@k across all users
    """
    recalls = []
    for user_true, user_pred in zip(y_true, y_pred):
        # Get indices of rated items in true ratings
        relevant_items = np.where(user_true > 0)[0]

        # If user has no relevant items, skip
        if len(relevant_items) == 0:
            continue

        # Get top k predicted items
        recommended_items = np.argsort(user_pred)[::-1][:k]

        # Calculate recall for this user
        hits = np.isin(recommended_items, relevant_items)
        recall = np.sum(hits) / min(k, len(relevant_items))
        recalls.append(recall)

    return np.mean(recalls) if recalls else 0

def calculate_ndcg(y_true, y_pred, k=5):
    """
    Calculate NDCG@k for each user.

    Args:
    y_true: True ratings (2D numpy array)
    y_pred: Predicted ratings (2D numpy array)
    k: Number of top items to consider

    Returns:
    Mean NDCG@k across all users
    """
    ndcgs = []
    for user_true, user_pred in zip(y_true, y_pred):
        # If user has no relevant items, skip
        if np.sum(user_true > 0) == 0:
            continue

        ndcg = ndcg_score(user_true.reshape(1, -1), user_pred.reshape(1, -1), k=k)
        ndcgs.append(ndcg)

    return np.mean(ndcgs) if ndcgs else 0

def hyperparameter_tuning(data, param_grid, test_size=0.2, random_state=42):
    X_train, X_test = train_test_split(data, test_size=test_size, random_state=random_state)
    best_params = None
    best_ndcg = float('-inf')
    for params in tqdm(ParameterGrid(param_grid), desc="Hyperparameter Tuning"):
        model = RBM(visible_units=data.shape[1], **params)
        model.fit(X_train)
        predictions = model.predict(X_test)

        ndcg = calculate_ndcg(X_test, predictions, k=10)
        recall = calculate_recall(X_test, predictions, k=10)
        precision = calculate_precision(X_test, predictions, k=10)
        map_score = calculate_map(X_test, predictions, k=10)

        log.info(f"Params: {params} - NDCG@10: {ndcg}, Recall@10: {recall}, Precision@10: {precision}, MAP@10: {map_score}")
        if ndcg > best_ndcg:
            best_ndcg = ndcg
            best_params = params
            best_recall = recall
            best_precision = precision
            best_map = map_score
    log.info(f"Best Params: {best_params} - Best NDCG@10: {best_ndcg}, Best Recall@10: {best_recall}, Best Precision@10: {best_precision}, Best MAP@10: {best_map}")
    return best_params, best_ndcg, best_recall, best_precision, best_map

def recommend(model, user_ratings, n=5):
    """
    Generate recommendations for a user.

    Args:
    model: Trained RBM model
    user_ratings: numpy array of user's ratings (1D array)
    n: Number of recommendations to generate

    Returns:
    List of indices of recommended items
    """
    # Reshape user_ratings to match the model input shape
    user_ratings = user_ratings.reshape(1, -1)

    # Get predictions
    predictions = model.predict(user_ratings)

    # Get indices of items the user hasn't rated
    unrated_items = np.where(user_ratings[0] == 0)[0]

    # Sort the predictions for unrated items
    sorted_predictions = np.argsort(predictions[0][unrated_items])[::-1]

    # Return the top n recommendations
    return unrated_items[sorted_predictions[:n]]

# Main execution
if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO)

    # Load your DataFrame
    # Assuming df is your DataFrame with user-item ratings
    # df = pd.read_csv('your_data.csv')  # Uncomment this line if you need to load the data

    # Pivot the DataFrame to create a user-item matrix
    #user_item_matrix = df.pivot(index='user_id', columns='item_id', values='rating').fillna(0)
    user_item_matrix = df.fillna(0)
    user_item_matrix.index = user_item_matrix.index.astype(int)
    user_item_matrix.columns = user_item_matrix.columns.astype(str)


    # Convert to numpy array
    data = user_item_matrix.values

    param_grid = {
        'hidden_units': [100],
        'keep_prob': [0.9],
        'learning_rate': [0.005],
        'minibatch_size': [64, 100],
        'training_epoch': [75, 100]
    }

    best_params, best_ndcg, best_recall, best_precision, best_map = hyperparameter_tuning(data, param_grid)
    print(f"Best Params: {best_params}")
    print(f"Best NDCG@5: {best_ndcg}")
    print(f"Best Recall@5: {best_recall}")
    print(f"Best Precision@5: {best_precision}")
    print(f"Best MAP@5: {best_map}")

    # Train the final model with best parameters
    final_model = RBM(visible_units=data.shape[1], **best_params)
    final_model.fit(data)

    # Example: Generate recommendations for the first user
    user_ratings = data[0]
    recommendations = recommend(final_model, user_ratings, n=5)
    print(f"Top 5 recommendations for user 0: {recommendations}")

    # Map item indices back to item IDs
    item_ids = user_item_matrix.columns[recommendations]
    print(f"Recommended item IDs: {item_ids.tolist()}")

Hyperparameter Tuning: 100%|██████████| 4/4 [1:51:33<00:00, 1673.42s/it]


Best Params: {'hidden_units': 100, 'keep_prob': 0.9, 'learning_rate': 0.005, 'minibatch_size': 100, 'training_epoch': 75}
Best NDCG@5: 0.4473765333411958
Best Recall@5: 0.780980492009718
Best Precision@5: 0.1597259442787076
Best MAP@5: 0.33025482329398426


Training: 100%|██████████| 76/76 [27:44<00:00, 21.90s/it]

Top 5 recommendations for user 0: [215 221  89 194 161]
Recommended item IDs: ['sorting', 'list', 'data-structures', 'recursion', 'memory-management']
